In [39]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional

USER_ID = 353
PATH_DATA = '../data/user_sequences/'
EXTENSION_TEXT = '.txt'
PATH_USER_DATA = PATH_DATA + f'user_{USER_ID}_sequence' + EXTENSION_TEXT

data = pd.read_csv(PATH_USER_DATA, header=None)
data = data.to_numpy()[:,1:]  # Discard weekday, just for now
print(data[0:3,:])

[[38.7364443 -9.1407651]
 [38.7356124 -9.1396516]
 [38.7368552 -9.1396499]]


In [40]:
print(data.shape)
#X = data[0:-1,:]
#y = data[1:,:]

X = data
y = data

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
y = scaler.transform(y)

X_train = X[0:190,:]
y_train = y[0:190,:]

X_test = X[190:,:]
y_test = y[190:,:]

sequence_length = 3
train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=16)
test_generator = TimeseriesGenerator(X_test, y_test, length=3, batch_size=4)

(240, 2)


In [41]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(sequence_length, 2), return_sequences=True))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(2))
lstm_model.compile(optimizer='adam', loss='mse')

epochs = 100
lstm_history = lstm_model.fit(train_generator, epochs=epochs, verbose=1, batch_size=4)

Epoch 1/100
12/12 [==============================] - 4s 13ms/step - loss: 1.0277
Epoch 2/100
12/12 [==============================] - 0s 13ms/step - loss: 1.0189
Epoch 3/100
12/12 [==============================] - 0s 15ms/step - loss: 1.0122
Epoch 4/100
12/12 [==============================] - 0s 15ms/step - loss: 1.0025
Epoch 5/100
12/12 [==============================] - 0s 16ms/step - loss: 0.9852
Epoch 6/100
12/12 [==============================] - 0s 16ms/step - loss: 0.9645
Epoch 7/100
12/12 [==============================] - 0s 16ms/step - loss: 0.9334
Epoch 8/100
12/12 [==============================] - 0s 17ms/step - loss: 0.8935
Epoch 9/100
12/12 [==============================] - 0s 16ms/step - loss: 0.8650
Epoch 10/100
12/12 [==============================] - 0s 15ms/step - loss: 0.8313
Epoch 11/100
12/12 [==============================] - 0s 6ms/step - loss: 0.8035
Epoch 12/100
12/12 [==============================] - 0s 6ms/step - loss: 0.7868
Epoch 13/100
12/12 [=======

In [42]:
lstm_score = lstm_model.evaluate(test_generator, verbose=0)

print(lstm_score)

0.7513700723648071
